<a href="https://colab.research.google.com/github/EmmanuelUdeA/Logica2/blob/main/ArbolesLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# LABORATORIO 06 tema árboles LOGICA 2

# Realizado por Emmanuel Bustamante Valbuena  UdeA 2023-1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import bisect


In [ ]:
#Constructor de un árbol
class TreeNode1(object):

    def __init__(self, value=None):
        self.value = value
        self.children = []

    def add_child(self, node):
        self.children.append(node)
        return self

    def to_indented_string(self, level):
        s = " "*level*2 + str(self.value)+"\n"
        if len(self.children)>0:
            s += "".join([i.to_indented_string(level+1) for i in self.children])
        return s

    def __repr__(self):
        return self.to_indented_string(level=0)

In [ ]:
t = TreeNode1("a").\
        add_child(TreeNode1("b").\
            add_child(TreeNode1("c")).\
            add_child(TreeNode1("d").\
              add_child(TreeNode1("e")))).\
        add_child(TreeNode1("f")).\
        add_child(TreeNode1("g").\
            add_child(TreeNode1("h")).\
            add_child(TreeNode1("i").\
              add_child(TreeNode1("j")).
              add_child(TreeNode1("k").\
                add_child(TreeNode1("l")))).\
            add_child(TreeNode1("m")).\
            add_child(TreeNode1("n")))
t

a
  b
    c
    d
      e
  f
  g
    h
    i
      j
      k
        l
    m
    n

In [ ]:
class TreeNode2(TreeNode1):

    def count_leaves(self):
        if len(self.children)==0:
            return 1
        return sum([i.count_leaves() for i in self.children])

    def height(self):
        import numpy as np
        if len(self.children)==0:
            return 0
        return np.max([i.height() for i in self.children])+1

    def grade(self):
        import numpy as np
        return np.max([len(self.children)]+[i.grade() for i in self.children])

In [ ]:
t = TreeNode2(1).add_child(TreeNode2(11).add_child(TreeNode2(111)).add_child(TreeNode2(112))).add_child(TreeNode2(12))
print(t)
print("nb leaves", t.count_leaves()) #1,11,111,12 (solo se toma un elemento de la subcolumna)
print("height   ", t.height()) # altura
print("grade    ", t.grade())  #

In [ ]:
t = TreeNode2("a").\
        add_child(TreeNode2("b").\
            add_child(TreeNode2("c")).\
            add_child(TreeNode2("d").\
              add_child(TreeNode2("e")))).\
        add_child(TreeNode2("f")).\
        add_child(TreeNode2("g").\
            add_child(TreeNode2("h")).\
            add_child(TreeNode2("i").\
              add_child(TreeNode2("j")).
              add_child(TreeNode2("k").\
                add_child(TreeNode2("l")))).\
            add_child(TreeNode2("m")).\
            add_child(TreeNode2("n")))
t

In [ ]:
print("nb leaves", t.count_leaves())
print("height   ", t.height())
print("grade    ", t.grade())

In [ ]:
# ---------- REPRESENTACION DE UN ARBOL EN UN VECTOR ---------------- #

In [ ]:
import numpy as np
class VBinTree(object):

    def __init__(self, v):
        self.v = v

    def get_height(self):
        i,c=0,0
        while c<len(self.v):
            c += 2**i
            i += 1
        return i-1

    def get_level(self, i):
        n = 0
        while sum([2**j for j in range(n)])<=i:
            n+=1
        return n-1

    def get_children_positions(self, i):
        return (2*i+1 if 2*i+1<len(self.v) else None,\
                2*i+2 if 2*i+2<len(self.v) else None)

    def get_parent_position(self, i):
        assert type(i)==int and i>=0
        return (i-1)/2 if i!=0 else None

    def get_sibling_position(self, i):
        if i==0:
            return None
        return [k for k in self.get_children_positions(self.get_parent_position(i)) if k!=i][0]


    def to_indented_string(self, i, level):
        c = self.get_children_positions(i)
        s = (" "*2*level + str(self.v[i]) + "\n") if self.v[i] is not None else ""
        s += self.to_indented_string(c[0],level+1) if c[0] is not None else ""
        s += self.to_indented_string(c[1],level+1) if c[1] is not None else ""
        return s

    def __repr__(self):
        return self.to_indented_string(0,0)

In [ ]:
v = ["a", "b", "i", "c", "d", "j", "k", "e", "f", "g", "h", "l", "m", "n", "o"]

b = VBinTree(v)

print("  node value  height     children parent")
for i in range(len(v)):
    print("%5d"%i, "%5s"%str(b.v[i]), "%5d"%b.get_level(i), \
          "%15s"%str(b.get_children_positions(i)), \
          "%5s"%str(b.get_parent_position(i)), \
          str(b.get_sibling_position(i)))

In [ ]:
# ARBOLES BINARIOS #

In [ ]:
class BTNode:

    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left  = None
        self.right = None
        if left is not None:
            self.add_left(left)
        if right is not None:
            self.add_right(right)

    def add_left(self, value):
        assert self.left is None, "node already has left child"
        self.left  = self.__class__(value) if not isinstance(value,self.__class__) else value
        return self

    def add_right(self, value):
        assert self.right is None, "node already has right child"
        self.right  = self.__class__(value) if not isinstance(value,self.__class__) else value
        return self

    def swap_children(self):
        tmp = self.left
        self.left = self.right
        self.right = self.left
        return self

    def insert_ordered(self, new_value):
        if new_value < self.value:
            if self.left is None:
                self.add_left(new_value)
            else:
                self.left.insert_ordered(new_value)
        else:
            if self.right is None:
                self.add_right(new_value)
            else:
                self.right.insert_ordered(new_value)
        return self

    def ird(self):
        if self.value==None:
            return []
        s1 = self.left.ird() if self.left is not None else []
        s2 = self.right.ird() if self.right is not None else []
        return s1+[self.value]+s2

    def to_indented_string(self, level, prefix=""):
        s = (" "*2*level + prefix + str(self.value) + "\n") if self.value is not None else ""
        s += self.left.to_indented_string(level+1, prefix="L: ") if self.left is not None else ""
        s += self.right.to_indented_string(level+1, prefix="R: ") if self.right is not None else ""
        return s

    def __repr__(self):
        return self.to_indented_string(0)

    @classmethod
    def from_list(cls, a_list):
        r = cls(a_list[0])
        for i in a_list[1:]:
            r.insert_ordered(i)
        return r

    @classmethod
    def sort_list(cls, a_list):
        r = cls.from_list(a_list)
        return np.r_[r.ird()]

In [ ]:
# -------------- Laboratorio 6 ARBOLES ------------------------ #

In [ ]:
# 1) implementar metodo shift_down

def VBinTree1(v=None, **kwargs):

    import numpy as np

    class VBinTree(object):

        def __init__(self, v=v,**kwargs):
            import numpy as np
            self.v = np.r_[[i for i in v]] # copy

        def get_height(self):
            i,c=0,0
            while c<len(self.v):
                c += 2**i
                i += 1
            return i-1

        def get_level(self, i):
            n = 0
            while sum([2**j for j in range(n)])<=i:
                n+=1
            return n-1

        def get_children_positions(self, i):
            return (2*i+1 if 2*i+1<len(self.v) else None,\
                    2*i+2 if 2*i+2<len(self.v) else None)

        def get_parent_position(self, i):
            assert type(i)==int and i>=0
            return (i-1)/2 if i!=0 else None

        def get_sibling_position(self, i):
            if i==0:
                return None
            return [k for k in self.get_children_positions(self.get_parent_position(i)) if k!=i][0]


        def to_indented_string(self, i, level):
            c = self.get_children_positions(i)
            s = (" "*2*level + str(self.v[i]) + "\n") if self.v[i] is not None else ""
            s += self.to_indented_string(c[0],level+1) if c[0] is not None else ""
            s += self.to_indented_string(c[1],level+1) if c[1] is not None else ""
            return s

        def __repr__(self):
            return self.to_indented_string(0,0)


    class VBinTree1_class(VBinTree):

        def shift_down(self, start, end=None):


          if end is None:
            end = len(self.v) - 1 # Verificamos que no sea un arbol vacio
 # ----------- pseudocodigo ------------ #
          pos_nodo = start

          while pos_nodo * 2 + 1 <= end:  # Mientras el nodo tenga hijos y no se pase de la posición end
                hijos = self.get_children_positions(pos_nodo)
                hijo_izquierdo = hijos[0]
                hijo_derecho = hijos[1]
                mayor_hijo = None

                if hijo_izquierdo is not None:
                    mayor_hijo = hijo_izquierdo

                if hijo_derecho is not None and self.v[hijo_derecho] > self.v[hijo_izquierdo]:
                    # Si el hijo derecho existe y es mayor que el hijo izquierdo
                    mayor_hijo = hijo_derecho

                if mayor_hijo is not None and self.v[mayor_hijo] > self.v[pos_nodo]:
                    # Si hay un hijo mayor y es mayor que el nodo actual, se intercambian
                    self.v[pos_nodo], self.v[mayor_hijo] = self.v[mayor_hijo], self.v[pos_nodo]
                    pos_nodo = mayor_hijo
                else:
                    break

          return self


    return VBinTree1_class(**kwargs)

In [ ]:
v = [5,9,4,1,6,2,8,3]
print (VBinTree1(v))
print (VBinTree1(v).shift_down(0))
print (VBinTree1(v).shift_down(1))
print (VBinTree1(v).shift_down(3))

5
  9
    1
      3
    6
  4
    2
    8

9
  6
    1
      3
    5
  4
    2
    8

5
  9
    1
      3
    6
  4
    2
    8

5
  9
    3
      1
    6
  4
    2
    8



In [ ]:
v = [5,9,4,1,6,2,8,3]
b = VBinTree1(v)
print(b)
i=3
print("shifting down element",i,"whose value is", b.v[i])
b.shift_down(i)
print(b)

5
  9
    1
      3
    6
  4
    2
    8

shifting down element 3 whose value is 1
5
  9
    3
      1
    6
  4
    2
    8



In [ ]:
# 2) implementar metodo get_last_parent_position(self):

def VBinTree2(v=None,**kwargs):
    import numpy as np

    class VBinTree(object):

        def __init__(self, v=v,**kwargs):
            import numpy as np
            self.v = np.r_[[i for i in v]] # copy

        def get_height(self):
            i,c=0,0
            while c<len(self.v):
                c += 2**i
                i += 1
            return i-1

        def get_level(self, i):
            n = 0
            while sum([2**j for j in range(n)])<=i:
                n+=1
            return n-1

        def get_children_positions(self, i):
            return (2*i+1 if 2*i+1<len(self.v) else None,\
                    2*i+2 if 2*i+2<len(self.v) else None)

        def get_parent_position(self, i):
            assert type(i)==int and i>=0
            return (i-1)/2 if i!=0 else None

        def get_sibling_position(self, i):
            if i==0:
                return None
            return [k for k in self.get_children_positions(self.get_parent_position(i)) if k!=i][0]


        def to_indented_string(self, i, level):
            c = self.get_children_positions(i)
            s = (" "*2*level + str(self.v[i]) + "\n") if self.v[i] is not None else ""
            s += self.to_indented_string(c[0],level+1) if c[0] is not None else ""
            s += self.to_indented_string(c[1],level+1) if c[1] is not None else ""
            return s

        def __repr__(self):
            return self.to_indented_string(0,0)

    class VBinTree2_class(VBinTree):

        def get_last_parent_position(self):
            pos_nodo = len(self.v) - 1  # Comenzar desde la posición del último nodo en el vector

            while pos_nodo >= 0:
                hijos = self.get_children_positions(pos_nodo)
                if hijos[0] is not None or hijos[1] is not None:
                    # Si el nodo tiene al menos un hijo, es el último padre en el árbol
                    return pos_nodo
                pos_nodo -= 1

            return None



    return VBinTree2_class(**kwargs)

In [ ]:
for v in [[5,9,0,1,6,2,8,3,4,5], [5,9,0,1,6,2,8]]:
    b = VBinTree2(v)
    print("\n\ntree:")
    print(b)
    p = b.get_last_parent_position()
    print("last parent position",p, "content", b.v[p])



tree:
5
  9
    1
      3
      4
    6
      5
  0
    2
    8

last parent position 4 content 6


tree:
5
  9
    1
    6
  0
    2
    8

last parent position 2 content 0


In [ ]:
#-----------------------#

In [ ]:
# 3) Completar metodo make_heap

def VBinTree3(v=None,**kwargs):

    import numpy as np

    class VBinTree(object):

        def __init__(self, v=v,**kwargs):
            import numpy as np
            self.v = np.r_[[i for i in v]] # copy

        def get_height(self):
            i,c=0,0
            while c<len(self.v):
                c += 2**i
                i += 1
            return i-1

        def get_level(self, i):
            n = 0
            while sum([2**j for j in range(n)])<=i:
                n+=1
            return n-1

        def get_children_positions(self, i):
            return (2*i+1 if 2*i+1<len(self.v) else None,\
                    2*i+2 if 2*i+2<len(self.v) else None)

        def get_parent_position(self, i):
            assert type(i)==int and i>=0
            return (i-1)/2 if i!=0 else None

        def get_sibling_position(self, i):
            if i==0:
                return None
            return [k for k in self.get_children_positions(self.get_parent_position(i)) if k!=i][0]


        def to_indented_string(self, i, level):
            c = self.get_children_positions(i)
            s = (" "*2*level + str(self.v[i]) + "\n") if self.v[i] is not None else ""
            s += self.to_indented_string(c[0],level+1) if c[0] is not None else ""
            s += self.to_indented_string(c[1],level+1) if c[1] is not None else ""
            return s

        def __repr__(self):
            return self.to_indented_string(0,0)

    class VBinTree3_class(VBinTree):

        def shift_down(self, start, end=None):


          if end is None:
            end = len(self.v) - 1 # Verificamos que no sea un arbol vacio

          pos_nodo = start

          while pos_nodo * 2 + 1 <= end:  # Mientras el nodo tenga hijos y no se pase de la posición end
                hijos = self.get_children_positions(pos_nodo)
                hijo_izquierdo = hijos[0]
                hijo_derecho = hijos[1]
                mayor_hijo = None

                if hijo_izquierdo is not None:
                    mayor_hijo = hijo_izquierdo

                if hijo_derecho is not None and self.v[hijo_derecho] > self.v[hijo_izquierdo]:
                    # Si el hijo derecho existe y es mayor que el hijo izquierdo
                    mayor_hijo = hijo_derecho

                if mayor_hijo is not None and self.v[mayor_hijo] > self.v[pos_nodo]:
                    # Si hay un hijo mayor y es mayor que el nodo actual, se intercambian
                    self.v[pos_nodo], self.v[mayor_hijo] = self.v[mayor_hijo], self.v[pos_nodo]
                    pos_nodo = mayor_hijo
                else:
                    break

          return self

        def get_last_parent_position(self):
            pos_nodo = len(self.v) - 1  # Comenzar desde la posición del último nodo en el vector

            while pos_nodo >= 0:
                hijos = self.get_children_positions(pos_nodo)
                if hijos[0] is not None or hijos[1] is not None:
                    # Si el nodo tiene al menos un hijo, es el último padre en el árbol
                    return pos_nodo
                pos_nodo -= 1

            return None

        def make_heap(self):

            last_parent = self.get_last_parent_position() # obtenemos la posicion del último padre

            if last_parent is not None: # verificar que exista al menos 1
             while last_parent >= 0: # iterar desde el último padre hasta el primer nodo del árbol
                self.shift_down(last_parent)
                last_parent -= 1




    return VBinTree3_class(**kwargs)

In [ ]:
import numpy as np
v = list(np.random.permutation(100)[:10])
v = [2,48,65,95,66,42,30,32]
b = VBinTree3(v)
print("-- original tree")
print(b)
print("-- constructed heap")
b.make_heap()
print()
print(b)

-- original tree
2
  48
    95
      32
    66
  65
    42
    30

-- constructed heap

95
  66
    48
      32
    2
  65
    42
    30



In [ ]:
#-----------------------------#

In [ ]:
#
# 4) implementar método sort que nos organiza el arbol de manera ascendente
def VBinTree4(v=None,**kwargs):

    class VBinTree(object):

        def __init__(self, v=v,**kwargs):
            import numpy as np
            self.v = np.r_[[i for i in v]] # copy

        def get_height(self):
            i,c=0,0
            while c<len(self.v):
                c += 2**i
                i += 1
            return i-1

        def get_level(self, i):
            n = 0
            while sum([2**j for j in range(n)])<=i:
                n+=1
            return n-1

        def get_children_positions(self, i):
            return (2*i+1 if 2*i+1<len(self.v) else None,\
                    2*i+2 if 2*i+2<len(self.v) else None)

        def get_parent_position(self, i):
            assert type(i)==int and i>=0
            return (i-1)/2 if i!=0 else None

        def get_sibling_position(self, i):
            if i==0:
                return None
            return [k for k in self.get_children_positions(self.get_parent_position(i)) if k!=i][0]


        def to_indented_string(self, i, level):
            c = self.get_children_positions(i)
            s = (" "*2*level + str(self.v[i]) + "\n") if self.v[i] is not None else ""
            s += self.to_indented_string(c[0],level+1) if c[0] is not None else ""
            s += self.to_indented_string(c[1],level+1) if c[1] is not None else ""
            return s

        def __repr__(self):
            return self.to_indented_string(0,0)


    class VBinTree4_class(VBinTree):

        def shift_down(self, start, end=None):
          if end is None:
            end = len(self.v) - 1

          pos_nodo = start

          while pos_nodo * 2 + 1 <= end:  # Mientras el nodo tenga hijos y no se pase de la posición end
            hijo_izquierdo = pos_nodo * 2 + 1
            hijo_derecho = pos_nodo * 2 + 2
            mayor_hijo = None

            if hijo_izquierdo <= end:
                mayor_hijo = hijo_izquierdo

            if hijo_derecho <= end and self.v[hijo_derecho] > self.v[hijo_izquierdo]:
                # Si el hijo derecho existe y es mayor que el hijo izquierdo
                mayor_hijo = hijo_derecho

            if mayor_hijo is not None and self.v[mayor_hijo] > self.v[pos_nodo]:
                # Si hay un hijo mayor y es mayor que el nodo actual, se intercambian
                self.v[pos_nodo], self.v[mayor_hijo] = self.v[mayor_hijo], self.v[pos_nodo]
                pos_nodo = mayor_hijo
            else:
                break

          return self

        def get_last_parent_position(self):
          pos_nodo = len(self.v) - 1

          while pos_nodo >= 0:
              hijo_izquierdo = pos_nodo * 2 + 1
              hijo_derecho = pos_nodo * 2 + 2

              if hijo_derecho < len(self.v) or hijo_izquierdo < len(self.v):
                  # Si el nodo tiene al menos un hijo, se considera padre
                  break

              pos_nodo -= 1

          return pos_nodo

        def make_heap(self):
            last_parent = self.get_last_parent_position() # obtenemos la posicion del último padre

            if last_parent is not None: # verificar que exista al menos 1
             while last_parent >= 0: # iterar desde el último padre hasta el primer nodo del árbol
                self.shift_down(last_parent)
                last_parent -= 1

            return self

        def sort(self):
            self.make_heap() # paso 1 construir
            last_node = len(self.v) - 1 # último nodo del arbol

            while last_node > 0:
                # Paso 2: Intercambiar valor de la raíz con nodo
                self.v[0], self.v[last_node] = self.v[last_node], self.v[0]

                # Paso 2: Shift_down de la raíz hasta nodo-1
                self.shift_down(0, last_node - 1)

                last_node -= 1 # paso 3 reducir para pasar al siguiente nodo

            return self # devuelve el arbol

    return VBinTree4_class(**kwargs)

In [ ]:
import numpy as np

v = list(np.random.permutation(100)[:15])
v = [57,58,73,48,62,75,88,15,29,40,53,61,56,8,1,32,51]
b = VBinTree4(v)
print("-- original tree")
print(b.v)

b.sort()
print("-- sorted tree")
print(b.v)

-- original tree
[57 58 73 48 62 75 88 15 29 40 53 61 56  8  1 32 51]
-- sorted tree
[ 1  8 15 29 32 40 48 51 53 56 57 58 61 62 73 75 88]


In [ ]:
import numpy as np

r=[]
n=1000
lengths = []
for i in range(n):
    v = list(np.random.permutation(1000)[:np.random.randint(300)])
    b = VBinTree4(v)
    b.sort()
    lengths.append(len(v))

    a=np.r_[b.v]
    r.append(np.alltrue(a[1:]-a[:-1]>0))

In [ ]:
print("número total de vectores generados:", n)
print("número total de vectores ordenados:", np.sum(r))
print("avg/std longitud de los vectores:  ", np.mean(lengths), "+/- %.1f"%np.std(lengths))

número total de vectores generados: 1000
número total de vectores ordenados: 1000
avg/std longitud de los vectores:   150.933 +/- 85.3
